In [1]:
import mysql.connector
import json
import time
def get_connection():
    return mysql.connector.connect(
        host="kadman-database.c8boumioc8hx.us-east-1.rds.amazonaws.com",
        user="admin",
        password="12345678",
        database="kadmandb",
        port=3306,
        charset='utf8mb4'
    )

def wait_for_db_connection():
    while True:
        try:
            conn = get_connection()
            print("✅ Successfully connected to MySQL database")
            return conn
        except Exception as e:
            print(f"❌ Failed to connect to DB: {e}. Retrying in 1 seconds...")
            time.sleep(1)

In [2]:
con = wait_for_db_connection()
cursor = con.cursor()


✅ Successfully connected to MySQL database


In [ ]:
# Query all vendor data
cursor.execute("ALTER TABLE vendors DROP COLUMN lineID")
rows = cursor.fetchall()
print(rows)

In [16]:
cursor.execute("ALTER TABLE vendors ADD COLUMN UserProfile VARCHAR(20)")
con.commit()
cursor.close()
con.close()

In [6]:
cursor = con.cursor()
cursor.execute("SELECT * FROM layouts")

# Get all rows
rows = cursor.fetchall()

# Get column names
column_names = [desc[0] for desc in cursor.description]

# Convert rows to list of dicts
result = [dict(zip(column_names, row)) for row in rows]

json_output = json.dumps(result, ensure_ascii=False, indent=2)
# Optional: Save to file
with open("layouts_dump.json", "w", encoding="utf-8") as f:
    f.write(json_output)

In [54]:
conn = wait_for_db_connection()

try:
    cursor
except NameError:
    cursor = None

if cursor is not None:
    try:
        # fetch all unread results if any
        cursor.fetchall()
    except Exception:
        # ignore if no results or error
        pass
    cursor.close()
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS waiting_vendors")
print("✅ Table 'waiting_vendors' dropped successfully.")
cursor.close()
conn.commit()
cursor.close()


✅ Successfully connected to MySQL database
✅ Table 'waiting_vendors' dropped successfully.


False

In [2]:
def create_waiting_vendors_table():
    conn = get_connection()
    cursor = conn.cursor()

    create_table_sql = """
    CREATE TABLE IF NOT EXISTS waiting_vendors (
        LineID VARCHAR(255) COLLATE utf8mb4_unicode_ci NOT NULL,
        UserProfile VARCHAR(255) COLLATE utf8mb4_unicode_ci NOT NULL,
        PRIMARY KEY (LineID)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
    """

    try:
        cursor.execute(create_table_sql)
        print("✅ Table 'waiting_vendors' created successfully.")
    except mysql.connector.Error as err:
        print(f"❌ Error creating table: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
create_waiting_vendors_table()


✅ Table 'waiting_vendors' created successfully.


In [2]:
def show_all_rows():
    conn = get_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT * FROM vendors")
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]  # Get column names

        if rows:
            print(f"Rows in 'vendors' ({len(rows)} rows):\n")
            print(" | ".join(columns))  # Print column headers
            print("-" * (len(" | ".join(columns)) + 10))
            for row in rows:
                print(" | ".join(str(cell) for cell in row))
        else:
            print("No rows found in table.")
    except mysql.connector.Error as err:
        print(f"Error fetching rows: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
show_all_rows()
    

Rows in 'vendors' (113 rows):

vendorID | vendor_name | shop_name | lineID | badges | attendance
---------------------------------------------------------------------------
1 | None | อิงสร้อยผ้าโพก | None | ["เครื่องประดับ"] | 3
2 | None | โอเสื้อยืด,เชิ้ต,แว่นตา | None | [] | 3
3 | None | ไอซ์ stronger.store25 เสื้อยืดแจ็คเก็ต | None | ["เสื้อผ้า"] | 3
4 | None | พี่ตั้มหอยทากเสื้อยืดของเล่น | None | ["เสื้อผ้า", "ของเล่น"] | 3
5 | None | แม่ต่ายเสื้อยืด | None | ["เสื้อผ้า", "ของเล่น"] | 3
6 | None | แม่ต่ายแขนยาว | None | ["เสื้อผ้า"] | 3
7 | None | โอมเพี้ยงเสื้อยืด | None | ["เสื้อผ้า"] | 3
8 | None | พี่เบนซ์กระเป๋า | None | ["กระเป๋า"] | 3
9 | None | ฟ่าง y2k | None | ["เสื้อผ้า", "y2k"] | 3
10 | None | อาย daffodil.3store เสื้อฮิปปี้ | None | ["เสื้อผ้า"] | 3
11 | None | บับเบิ้ลเสื้อยืด เนคไท ผ้าเช็ดหน้า | None | ["เสื้อผ้า"] | 3
12 | None | พี่อาร์มเสื้อกีฬา | None | ["เสื้อผ้า"] | 3
13 | None | พี่ฮัทถุงเท้า | None | ["ถุงเท้า"] | 3
14 | None | พี่อ๋องแจ็คเก็ต | None | ["เส

In [12]:
def update_attendance_to_three():
    conn = get_connection()
    cursor = conn.cursor()

    try:
        cursor.execute("UPDATE vendors SET attendance = 3")
        conn.commit()
        print("All rows updated: attendance set to 3.")
    except mysql.connector.Error as err:
        print(f"Error updating rows: {err}")
    finally:
        cursor.close()
        conn.close()

# Run the function
update_attendance_to_three()


Error updating rows: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction


In [ ]:
sql = 'SHOW COLUMNS FROM kadmandb.columns'